In [1]:
import numpy as np
import pandas as pd
#Method 1 Rayleigh_Quotient_iteration
def Rayleigh_Quotient_iteration(A, v0):
    # get x=(A-mu*I).Inv*x0/||(A-mu*I).Inv*x0||
    x_prev = v0
    iteration = 0
    I = np.identity(A.shape[0])

    err = 1
    lam_i = (x_prev.T @ A @ x_prev) / (x_prev.T @ x_prev)
    #iterations range is 1 to 20
    for i in range(20):

        iteration = iteration + 1
        y = np.linalg.inv(A - lam_i * I) @ x_prev
        #eigenvector with normalization
        x = y / np.linalg.norm(y, ord=np.inf)
        #eigenvalue by using rayleigh quotient
        lam = (x_prev.T @ A @ x_prev) / (x_prev.T @ x_prev)
        #apply:err=(x1-x0)/x1
        err = np.linalg.norm((A @ x - lam * x) / (A @ x))
        #tolerance 0.0001
        if err < 0.0001:
            break
        x_prev = x
        lam_i = lam

        #print(iteration)
    return x, lam, iteration


A = np.array([[2.9766, 0.3945, 0.4198, 1.1159],
              [0.3945, 2.7328, -0.3097, 0.1129],
              [0.4198, -0.3097, 2.5675, 0.6079],
              [1.1159, 0.1129, 0.6079, 1.7231]])
print(A)
data=[]
#run for four different assume vectors
for i in range(4):
    v0 = np.zeros(4)
    v0[i] = 1
    v = v0.copy()
    v0 = v0.reshape(-1, 1)
    #print(v0)

    eigenvector, eigenvalue, iterations = Rayleigh_Quotient_iteration(A, v0)
    data.append([i, v0.T, eigenvector.T, eigenvalue, iterations])

df=pd.DataFrame(data,columns=['#','Starting Vector','Eigenvector','Eigenvalue','Num. of Iterations'])
df.to_csv('eigentable.csv',index=False)

[[ 2.9766  0.3945  0.4198  1.1159]
 [ 0.3945  2.7328 -0.3097  0.1129]
 [ 0.4198 -0.3097  2.5675  0.6079]
 [ 1.1159  0.1129  0.6079  1.7231]]


In [218]:
#Method QR_iteration:Gram_Schmidt
def Gram_Schmidt_QR_factor(A):
    r=A.shape[0]
    n=A.shape[1]
    q=A.copy()
    r=np.identity(n)
    for k in range(n):
        #qk=a[:,k]
        for j in range(k):
            r[j,k]=q[:,j].T@A[:,k]
            q[:,k]=q[:,k]-r[j,k]*q[:,j]
        #normalized q[:,k]
        r[k,k]=np.linalg.norm(q[:,k],2)
        #print(f'{k}time:{r[k,k]}')
        if r[k,k] ==0:
            continue

        q[:,k]=q[:,k]/r[k,k]
    #print(q)
    return q,r
q,r=Gram_Schmidt_QR_factor(A)
def Q_R_Iteration(A):
#use Q@R=Ak, Ak+1=R@Q
    for i in range(20):
        Q,R=Gram_Schmidt_QR_factor(A)
        A_new=np.dot(R,Q)
        #obtian the err
        old_eign=np.around(np.diagonal(A),decimals=6)
        new_eign=np.around(np.diagonal(A_new),decimals=6)

        err=np.linalg.norm((new_eign-old_eign)/new_eign)
        #break condition
        if err<0.0001:
            break
        #loop back to A and old_eign
        A=A_new
        old_eign=new_eign
    return new_eign,i


new_eign,i=Q_R_Iteration(A)
print(f'A = {A}')
print(f'The eigen values are: {new_eign}')
print(f'The number of iterations for the convergence is {i}')

A = [[ 2.9766  0.3945  0.4198  1.1159]
 [ 0.3945  2.7328 -0.3097  0.1129]
 [ 0.4198 -0.3097  2.5675  0.6079]
 [ 1.1159  0.1129  0.6079  1.7231]]
The eigen values are: [3.999965 2.999936 2.000115 0.999984]
The number of iterations for the convergence is 10
